In [ ]:
!pip3 install 'import_ipynb'
!pip3 install 'tqdm'

Import



In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import math
from sklearn.preprocessing import normalize
import copy
import torchvision.datasets as dsets
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')
  
%cd /content/drive/My\ Drive/MLDLProjects/Lande_Napolitano_Pipoli/Project_root
 
import import_ipynb
from cifar100 import cifar_100
import net
from net import resnet32
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1cjI5lckXdsY9KVNaWBpL_I91maSSOLBA/MLDLProjects/Lande_Napolitano_Pipoli/Project_root
importing Jupyter notebook from cifar100.ipynb
IMPORT CIFAR DONE


HyperParameters

In [ ]:
LR=2.0
GAMMA=0.2
WEIGHT_DECAY=1e-5
MOMENTUM=0.9
MILESTONE=[49,64]
BATCH_SIZE=128
NUM_EPOCHS=20
TOTAL_CLASSES = 100
NUM_CLASSES = 10
K=2000
DEVICE="cuda"
os.environ['PYTHONWARNINGS'] = "ignore"

iCarl Model

In [ ]:
class iCaRLNet(nn.Module):
    def __init__(self, feature_size, n_classes, lr=LR, gamma=GAMMA, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY, milestone=MILESTONE, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS):
        # Network architecture
        super(iCaRLNet, self).__init__()
        self.net = resnet32(num_classes=n_classes)
        self.feature_extractor = self.net.get_fm_out
        self.lr = lr
        self.gamma = gamma
        self.weight_decay = weight_decay
        self.milestone = milestone
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.n_classes = 0
        self.n_known = 0
        self.feature_size=feature_size
        self.momentum=momentum
        self.loss = nn.BCEWithLogitsLoss()

        self.compute_means = True
        self.compute_feat = True
        self.exemplar_means = []
        self.exemplar_sets = []
        self.exemplar_feat = []
        self.exemplar_lab = []
        self.KNN = None
        self.compute_SVM = True
        self.compute_knn = True
        self.SVM=None
        self.PCA=None
        self.forest = None
        self.compute_forest = True


    def forward(self, x):

        self.net = self.net.cuda()
        return self.net.forward(x)


    def increment_classes(self, n):

        in_features = self.fc.in_features
        out_features = self.fc.out_features
        weight = self.fc.weight.data
        self.fc = nn.Linear(in_features, out_features + n, bias=False)
        self.fc.weight.data[:out_features] = weight
        self.n_classes += n


    def classify(self, x, cif): 
        
        if self.compute_means:
            exemplar_means = []
            for P_y in self.exemplar_sets:  #P_y list of indice
                features = np.zeros((0,self.feature_size))
                sub = Subset(cif, P_y)
                dl = torch.utils.data.DataLoader(sub, batch_size=self.batch_size,shuffle=False, num_workers=4) 
                with torch.no_grad():
                    for ind, ex, lab in dl:# Extract feature for each exemplar in P_y
                        ex = Variable(ex).cuda()
                        feature = self.feature_extractor(ex).data.cpu().numpy()
                        feature = normalize(feature, axis=1, norm='l2')
                        features = np.concatenate((features,feature), axis=0)

                features = torch.tensor(features)
                mu_y = features.mean(0).squeeze()
                mu_y.data = mu_y.data / torch.norm(mu_y, p=2)  # L2 Normalize
                exemplar_means.append(mu_y)
                
            self.exemplar_means = exemplar_means
            self.compute_means = False

        exemplar_means = self.exemplar_means
        means = torch.stack(exemplar_means)  # (n_classes, feature_size)
        means = torch.stack([means] * self.batch_size)  # (batch_size, n_classes, feature_size)
        means = means.transpose(1, 2)  # (batch_size, feature_size, n_classes)

        feature = self.feature_extractor(x)  # (batch_size, feature_size)
        for i in range(feature.size(0)):  # Normalize
            feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        feature = feature.unsqueeze(2)  # (batch_size, feature_size, 1)
        feature = feature.expand_as(means)  # (batch_size, feature_size, n_classes)
        feature = feature.cuda()
        means = means.cuda()

        dists = torch.sqrt((feature - means).pow(2).sum(1)).squeeze()  # (batch_size, n_classes)
        _, preds = dists.min(1) #prev

        return preds


    def classify_CS(self, x, cif): 
        
        if self.compute_means:
            exemplar_means = []
            for P_y in self.exemplar_sets:  #P_y list of indice
                features = np.zeros((0,self.feature_size))
                sub = Subset(cif, P_y)
                dl = torch.utils.data.DataLoader(sub, batch_size=self.batch_size,shuffle=False, num_workers=4) 
                with torch.no_grad():
                    for ind, ex, lab in dl:# Extract feature for each exemplar in P_y
                        ex = Variable(ex).cuda()
                        feature = self.feature_extractor(ex).data.cpu().numpy()
                        feature = normalize(feature, axis=1, norm='l2')
                        features = np.concatenate((features,feature), axis=0)

                features = torch.tensor(features)
                mu_y = features.mean(0).squeeze()
                mu_y.data = mu_y.data / torch.norm(mu_y, p=2)  # L2 Normalize
                exemplar_means.append(mu_y)
                
            self.exemplar_means = exemplar_means
            self.compute_means = False

        exemplar_means = self.exemplar_means

        feature = self.feature_extractor(x)  # (batch_size, feature_size)
        for i in range(feature.size(0)):  # Normalize
            feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        feature = feature.cuda()

        dists=[]
        for i in range(feature.size(0)):
            f=feature[i].squeeze(0)
            f=f.detach().cpu().numpy()
            f=np.expand_dims(f,axis=0)
            dist=[]
            for means in exemplar_means:
                m=np.array(means)
                m=np.expand_dims(m,axis=0)
                d=cosine_similarity(f, m)
                dist.append(d)
            dist=np.array(dist)
            dist=np.argmax(dist)
            dists.append(dist)

        dists=torch.tensor(dists)

        return dists

    def compute_exemplar_features(self, cif, mapper):
        if self.compute_feat:
            exemplar_feat = np.zeros((0,self.feature_size))
            labs=[]
            for exemplar_list in self.exemplar_sets:  #exemplar_list list of exemplar indexes
                features = np.zeros((0,self.feature_size))
                sub = Subset(cif, exemplar_list)
                dl = torch.utils.data.DataLoader(sub, batch_size=self.batch_size,shuffle=False, num_workers=4) 

                with torch.no_grad():
                    for ind, ex, lab in dl:# Extract feature for each exemplar in exemplar_list
                        ex = Variable(ex).cuda()
                        lab = [mapper[l.item()] for l in lab]
                        feature = self.feature_extractor(ex).data.cpu().numpy() #(batch_size, feature_size)
                        feature = normalize(feature, axis=1, norm='l2')
                        features = np.concatenate((features,feature), axis=0)
                        labs.extend(lab)
                        
                exemplar_feat=np.concatenate((exemplar_feat,features), axis=0)

            self.exemplar_lab=labs
            self.exemplar_feat = exemplar_feat
            self.compute_feat = False


    def classify_KNN(self, x): 

        if self.compute_knn:
            # TRAIN
            x_train = self.exemplar_feat
            y_train = self.exemplar_lab
            neigh = KNeighborsClassifier(n_neighbors=10)
            neigh.fit(x_train, y_train)
            self.KNN=neigh
            self.compute_knn = False
        
        #TEST
        feature = self.feature_extractor(x)  # (batch_size, feature_size)
        for i in range(feature.size(0)):  # Normalize
            feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        x_test=feature.detach().cpu().numpy()
            
        y_pred=self.KNN.predict(x_test)
        y_pred=torch.Tensor(y_pred)
        
        return y_pred


    def classify_SVM(self, x): 

        if self.compute_SVM:
            #TRAIN
            x_train = self.exemplar_feat
            y_train = self.exemplar_lab
            c = 1 
            rbf = SVC(kernel='rbf', C=c) #RBF-LINEAR
            rbf.fit(x_train, y_train)
            self.SVM = rbf
            self.compute_SVM = False
      
        #TEST
        feature = self.feature_extractor(x)  # (batch_size, feature_size)
        for i in range(feature.size(0)):  # Normalize
            feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        x_test=feature.detach().cpu().numpy()

            
        y_pred=self.SVM.predict(x_test)
        y_pred=torch.Tensor(y_pred)
        
        return y_pred

    def classify_forest(self, x):
        if self.compute_forest:
            #TRAIN
            x_train = self.exemplar_feat
            y_train = self.exemplar_lab
            c = 10 
            forest = RandomForestClassifier(n_estimators=100)
            forest.fit(x_train, y_train)
            
            self.forest = forest
            self.compute_forest = False
      
        #TEST
        feature = self.feature_extractor(x)  # (batch_size, feature_size)
        for i in range(feature.size(0)):  # Normalize
            feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        x_test = feature.detach().cpu().numpy()

            
        y_pred=self.forest.predict(x_test)
        y_pred=torch.Tensor(y_pred)
        
        return y_pred


    def construct_exemplar_set(self, images, m):
        
        features = np.zeros((0,self.feature_size))
        indices = np.zeros((0), dtype=int)
        dl = torch.utils.data.DataLoader(images, batch_size=self.batch_size,shuffle=False, num_workers=4)
        with torch.no_grad():
          for ind, img, lab in dl:
            x = Variable(img).cuda()
            feature = self.feature_extractor(x).data.cpu().numpy()
            feature = normalize(feature, axis=1, norm='l2')
            features = np.concatenate((features,feature), axis=0)
            indices = np.concatenate((indices,ind), axis=0)

        class_mean = np.mean(features, axis=0)
        class_mean = class_mean / np.linalg.norm(class_mean)  # Normalize

        exemplar_set = []
        exemplar_features = np.zeros((0,64))

        for k in range(1, int(m)+1):
            S = np.sum(exemplar_features, axis=0)
            phi = features
            mu = class_mean
            mu_p = 1.0 / k * (phi + S)
            mu_p = normalize(mu_p, axis=1, norm='l2')
            i = np.argmin(np.sqrt(np.sum((mu - mu_p) ** 2, axis=1)))
            exemplar_set.append(indices[i])
            addfeature =  np.expand_dims(features[i], axis=0)
            exemplar_features = np.concatenate((exemplar_features,addfeature), axis=0)

            #remove duplicates
            features = np.delete(features, i, 0)
            indices = np.delete(indices, i, 0)
            
        self.exemplar_sets.append(exemplar_set)
        

    def reduce_exemplar_sets(self, m):

        for y, P_y in enumerate(self.exemplar_sets):
            self.exemplar_sets[y] = P_y[:int(m)]


    def combine_dataset_with_exemplars(self, cifar):

        newindexes = []
        for y, P_y in enumerate(self.exemplar_sets):
            exemplar_images = P_y
            exemplar_labels = [y] * len(P_y)
            print(exemplar_images[0].shape)
            newindexes+=cifar.data_append(exemplar_images, exemplar_labels)
        return newindexes


    def exemplarIndexes(self):

        Indexes = []
        for P_y in self.exemplar_sets:
            Indexes += P_y
        return Indexes


    def update_representation(self, cifar, batchindexes, diz):

        prev_model = copy.deepcopy(self)
        prev_model = prev_model.eval().cuda()
        self.compute_means = True
        self.compute_feat = True
        self.compute_SVM = True
        self.compute_forest = True
        self.compute_knn = True

        self.n_classes += 10

        # Form combined training set
        newindexes = []
        if self.n_classes > 10:
            newindexes = self.exemplarIndexes() #IMPORTANT!!!!
        newindexes += list(batchindexes)
        
        reprdata = Subset(cifar, newindexes)

        loader = torch.utils.data.DataLoader(reprdata, batch_size=self.batch_size,shuffle=True, num_workers=4, drop_last=True)

        optimizer = optim.SGD(self.net.parameters(), lr=self.lr, weight_decay=self.weight_decay, momentum=self.momentum)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestone, gamma=self.gamma)

        for epoch in tqdm(range(self.num_epochs)):
            losses = []
            for indices, images, labels in loader:
                labels = torch.tensor([torch.tensor(diz[c.item()]) for c in labels])
                images = Variable(torch.FloatTensor(images)).cuda()
                labels = Variable(labels).cuda()
                optimizer.zero_grad()
                g = self.forward(images)
                
                y_hot = F.one_hot(labels, self.n_classes).float().cuda()
                
                if self.n_known > 0:
                    q = prev_model.forward(images)
                    q = torch.sigmoid(q)
                    target = torch.cat((q[:,:self.n_known], y_hot[:,self.n_known:self.n_classes]), dim=1)
                    loss = self.loss(g[:,:self.n_classes], target)
                else:
                    loss = self.loss(g[:,:self.n_classes], y_hot[:,:self.n_classes])

                losses.append(loss.item())
                loss.backward()
                optimizer.step()
            
            scheduler.step()


In [ ]:
def compute_acc(dl, model, mapper):
    total = 0.0
    correct = 0.0
    correct_knn=0.0
    correct_svm=0.0
    correct_forest=0.0
    correct_cs = 0.0
    for _, images, labels in dl:
        labels = torch.tensor([torch.tensor(mapper[c.item()]) for c in labels])
        images = Variable(images).cuda()
        preds = model.classify(images, cifarTrain)
        preds_knn=model.classify_KNN(images)
        preds_svm=model.classify_SVM(images) 
        preds_forest = model.classify_forest(images)
        preds_cs=model.classify_CS(images, cifarTrain)
        total = total + len(labels)
        correct += (preds.data.cpu() == labels).sum()
        correct_knn += (preds_knn.data.cpu() == labels).sum()
        correct_svm += (preds_svm.data.cpu() == labels).sum()
        correct_forest += (preds_forest.data.cpu() == labels).sum()
        correct_cs += (preds_cs.data.cpu() == labels).sum()
    acc = 100 * correct / total
    acc_knn = 100 * correct_knn / total
    acc_svm = 100 * correct_svm / total
    acc_forest = 100 * correct_forest / total
    acc_cs = 100 * correct_cs / total
    return acc, acc_knn, acc_svm, acc_forest, acc_cs

In [ ]:
cifarTrain = cifar_100(NUM_CLASSES, 'train',981)
cifarTest = cifar_100(NUM_CLASSES, 'test',981)
randomlist = cifarTrain.get_classes_list()
mapper = cifarTrain.get_dictionary()

Files already downloaded and verified
Files already downloaded and verified


Main

In [ ]:
icarl = iCaRLNet(64, TOTAL_CLASSES, num_epochs=70).cuda()    
#with open("04_classifiers_SVM_C1", mode='w') as file:
print('-' * 80)
vKNN=[]
vSVM=[]
vRF=[]
for s in range(10):

        print(f"\nITERATION: {(s+1)*10}/100")

        print("Loading training examples for classes", randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
        
        batchindexes = cifarTrain.getClassIndexes(randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
        batch = Subset(cifarTrain, batchindexes)

        testindexes = cifarTest.getClassIndexes(randomlist[0:s*NUM_CLASSES + NUM_CLASSES])
        test_set = Subset(cifarTest, testindexes)
        l=0
        for el in icarl.exemplar_sets:
            l+=len(el)
        print("Batch size train: {} - Batch size test: {}".format(len(batch)+l, len(test_set)))

        train_loader = torch.utils.data.DataLoader(batch, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True) #FALSE

        icarl.train()
        icarl.update_representation(cifarTrain, batchindexes, mapper)
        icarl.eval()

        m = K / icarl.n_classes

        icarl.reduce_exemplar_sets(m)

        # Construct exemplar sets for new classes
        for y in randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES]:
            imagesInd = cifarTrain.getClassIndexes([y])
            images = Subset(cifarTrain, imagesInd)
            icarl.construct_exemplar_set(images, m)

        icarl.n_known = icarl.n_classes

        icarl.compute_exemplar_features(cifarTrain, mapper)

        acc, acc_knn, acc_svm, acc_forest, acc_cs = compute_acc(train_loader,icarl,mapper)
        #print('Train Accuracy: {:.2f}, KNN Accuracy: {:.2f}, SVM Accuracy: {:.2f}, Forest Accurracy: {:.2f}, CS Accurracy: {:.2f}'.format(acc, acc_knn, acc_svm, acc_forest, acc_cs).rstrip(), file=file)
        print('Train Accuracy: {:.2f}, KNN Accuracy: {:.2f}, SVM Accuracy: {:.2f}, Forest Accurracy: {:.2f}, CS Accurracy: {:.2f}'.format(acc, acc_knn, acc_svm, acc_forest, acc_cs).rstrip())

        acc, acc_knn, acc_svm, acc_forest, acc_cs = compute_acc(test_loader,icarl,mapper)
        #print('Test Accuracy: {:.2f}, KNN Accuracy: {:.2f}, SVM Accuracy: {:.2f}, Forest Accurracy: {:.2f}, CS Accurracy: {:.2f}'.format(acc, acc_knn, acc_svm, acc_forest, acc_cs).rstrip(), file=file)
        print('Test Accuracy: {:.2f}, KNN Accuracy: {:.2f}, SVM Accuracy: {:.2f}, Forest Accurracy: {:.2f}, CS Accurracy: {:.2f}'.format(acc, acc_knn, acc_svm, acc_forest, acc_cs).rstrip())
        vKNN.append(acc_knn)
        vSVM.append(acc_svm)
        vRF.append(acc_forest)
        print("\n")
        print('-' * 80)
        print("\n")
        print('-' * 80)
        
print("KNN")
print(vKNN)
print("SVM")
print(vSVM)
print("RF")
print(vRF)

--------------------------------------------------------------------------------

ITERATION: 10/100
Loading training examples for classes [56, 99, 35, 94, 2, 85, 96, 62, 12, 34]
Batch size train: 5000 - Batch size test: 1000



Train Accuracy: 94.41, KNN Accuracy: 94.67, SVM Accuracy: 94.65, Forest Accurracy: 94.19, CS Accurracy: 94.41
Test Accuracy: 83.59, KNN Accuracy: 83.37, SVM Accuracy: 82.81, Forest Accurracy: 82.48, CS Accurracy: 83.59


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 20/100
Loading training examples for classes [19, 63, 40, 24, 79, 97, 92, 20, 7, 53]
Batch size train: 7000 - Batch size test: 2000



Train Accuracy: 93.01, KNN Accuracy: 94.65, SVM Accuracy: 95.57, Forest Accurracy: 92.45, CS Accurracy: 93.01
Test Accuracy: 74.64, KNN Accuracy: 75.52, SVM Accuracy: 75.10, Forest Accurracy: 74.22, CS Accurracy: 74.64


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 30/100
Loading training examples for classes [76, 39, 17, 0, 18, 27, 74, 9, 37, 3]
Batch size train: 7000 - Batch size test: 3000



Train Accuracy: 88.78, KNN Accuracy: 89.12, SVM Accuracy: 93.03, Forest Accurracy: 88.32, CS Accurracy: 88.78
Test Accuracy: 68.34, KNN Accuracy: 68.72, SVM Accuracy: 68.72, Forest Accurracy: 66.61, CS Accurracy: 68.34


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 40/100
Loading training examples for classes [45, 78, 65, 75, 16, 57, 83, 30, 1, 22]
Batch size train: 6980 - Batch size test: 4000



Train Accuracy: 85.62, KNN Accuracy: 83.71, SVM Accuracy: 90.62, Forest Accurracy: 84.80, CS Accurracy: 85.62
Test Accuracy: 62.58, KNN Accuracy: 62.20, SVM Accuracy: 63.03, Forest Accurracy: 60.13, CS Accurracy: 62.58


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 50/100
Loading training examples for classes [11, 8, 38, 15, 49, 87, 26, 42, 91, 61]
Batch size train: 7000 - Batch size test: 5000


...


Train Accuracy: 77.62, KNN Accuracy: 72.34, SVM Accuracy: 84.29, Forest Accurracy: 71.55, CS Accurracy: 77.62
Test Accuracy: 53.38, KNN Accuracy: 51.63, SVM Accuracy: 53.52, Forest Accurracy: 48.28, CS Accurracy: 53.38


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 70/100
Loading training examples for classes [5, 51, 14, 71, 88, 28, 41, 25, 80, 54]
Batch size train: 6980 - Batch size test: 7000



Train Accuracy: 80.37, KNN Accuracy: 73.18, SVM Accuracy: 85.48, Forest Accurracy: 77.38, CS Accurracy: 80.37
Test Accuracy: 50.62, KNN Accuracy: 48.48, SVM Accuracy: 50.55, Forest Accurracy: 44.27, CS Accurracy: 50.62


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 80/100
Loading training examples for classes [55, 32, 13, 52, 36, 86, 64, 60, 93, 77]
Batch size train: 6960 - Batch size test: 8000



Train Accuracy: 77.20, KNN Accuracy: 71.43, SVM Accuracy: 82.77, Forest Accurracy: 71.67, CS Accurracy: 77.20
Test Accuracy: 46.62, KNN Accuracy: 45.30, SVM Accuracy: 47.61, Forest Accurracy: 41.91, CS Accurracy: 46.62


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 90/100
Loading training examples for classes [67, 29, 21, 73, 46, 95, 68, 69, 33, 98]
Batch size train: 7000 - Batch size test: 9000



Train Accuracy: 77.32, KNN Accuracy: 69.79, SVM Accuracy: 82.25, Forest Accurracy: 75.08, CS Accurracy: 77.32
Test Accuracy: 44.03, KNN Accuracy: 41.88, SVM Accuracy: 44.51, Forest Accurracy: 39.05, CS Accurracy: 44.03


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

ITERATION: 100/100
Loading training examples for classes [58, 84, 47, 50, 72, 82, 59, 31, 48, 43]
Batch size train: 6980 - Batch size test: 10000



Train Accuracy: 76.46, KNN Accuracy: 67.61, SVM Accuracy: 83.11, Forest Accurracy: 71.43, CS Accurracy: 76.46
Test Accuracy: 42.21, KNN Accuracy: 40.08, SVM Accuracy: 42.82, Forest Accurracy: 37.55, CS Accurracy: 42.21


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
KNN
[tensor(83.3705), tensor(75.5208), tensor(68.7160), tensor(62.1976), tensor(56.4103), tensor(51.6304), tensor(48.4809), tensor(45.2999), tensor(41.8750), tensor(40.0841)]
SVM
[tensor(82.8125), tensor(75.1042), tensor(68.7160), tensor(63.0292), tensor(56.6907), tensor(53.5156), tensor(50.5498), tensor(47.6058), tensor(44.5089), tensor(42.8185)]
RF
[tensor(82.4777), tensor(74.2188), tensor(66.6101), tensor(60.1311), tensor(53.7260), tensor(48.2846), tensor(44.2708), tensor(41.9103), tensor(39.0513), tensor(37.5501)]


In [ ]:
clf=["KNN","SVM","RF"]
for i,vect in enumerate([vKNN, vSVM, vRF]):
    l=[]
    for el in vect:
        el=el.item()
        l.append(el)
    print(clf[i])
    print(l)

KNN
[83.37053680419922, 75.52083587646484, 68.71603393554688, 62.19758224487305, 56.410255432128906, 51.630435943603516, 48.48090362548828, 45.29990005493164, 41.875, 40.08413314819336]
SVM
[82.8125, 75.10416412353516, 68.71603393554688, 63.029232025146484, 56.690704345703125, 53.515625, 50.54977035522461, 47.6058464050293, 44.50893020629883, 42.81850814819336]
RF
[82.47767639160156, 74.21875, 66.61005401611328, 60.13105010986328, 53.72595977783203, 48.284645080566406, 44.27083206176758, 41.910282135009766, 39.05133819580078, 37.550079345703125]
